In [1]:
%matplotlib inline
import imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import warnings
import seaborn as sns; sns.set()
import os
import pandas_profiling
from haversine import haversine, Unit
pd.set_option('display.max_columns', None)
warnings.filterwarnings("ignore")

ModuleNotFoundError: No module named 'haversine'

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.base import BaseEstimator, ClassifierMixin, RegressorMixin, ClusterMixin
from sklearn.model_selection import train_test_split

In [3]:
# Feature extractor
import pandas as pd
import os

class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_array):
        pass

    def transform(self, X_df):
        X_encoded = X_df
        path = os.path.dirname(__file__)
        # Feature engineering
        # Creating columns to distinguish between the two main airports for flights and the rest
        X_encoded['d_ManyFlights'] = 0
        X_encoded['a_ManyFlights'] = 0
        X_encoded['d_ManyFlights'][X_df['Departure'] == 'ORD'] = 1
        X_encoded['d_ManyFlights'][X_df['Departure'] == 'ATL'] = 1
        X_encoded['a_ManyFlights'][X_df['Arrival'] == 'ORD'] = 1
        X_encoded['a_ManyFlights'][X_df['Arrival'] == 'ATL'] = 1

        # Categorical encoding of departure and arrival airports
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))

        # Categorical encoding of the dates 
        X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
        X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
        X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
        X_encoded['day'] = X_encoded['DateOfDeparture'].dt.day
        X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
        X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
        X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))



        # Auxiliary dataset we will use to complete our data
        external_dataTest= pd.read_csv(os.path.join(path, 'external_data.csv'))
        external_data = external_dataTest['Date', 'AirPort', 'Max TemperatureC', 'Mean TemperatureC',
                                          'Min TemperatureC', 'Dew PointC', 'MeanDew PointC', 'Min DewpointC',
                                          'Max Humidity', 'Mean Humidity', 'Min Humidity',
                                          'Max Sea Level PressurehPa', 'Mean Sea Level PressurehPa',
                                          'Min Sea Level PressurehPa', 'Max VisibilityKm', 'Mean VisibilityKm',
                                          'Min VisibilitykM', 'Max Wind SpeedKm/h', 'Mean Wind SpeedKm/h',
                                          'Max Gust SpeedKm/h', 'Precipitationmm', 'CloudCover', 'Events',
                                          'WindDirDegrees', 'Rain', 'Thunderstorm', 'Fog', 'Snow', 'Hail',
                                          'Tornado', 'latitude_deg', 'longitude_deg', 'elevation_ft',
                                          '2018', '2017', '2016', '2015']

        # Now merging external data with out dataset 
        # Creating two tables, one for departures, the other for arrivals
        external_dataDeparture = external_data.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Departure'})
        external_dataArrival = external_data.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})

        # Merging them with X_encoded   
        X_encoded = pd.merge(X_encoded, external_dataDeparture, how='left',left_on=['DateOfDeparture', 'Departure'],
                             right_on=['DateOfDeparture', 'Departure'],sort=False)
        X_encoded = pd.merge(X_encoded, external_dataArrival, how='left',left_on=['DateOfDeparture', 'Arrival'],
                             right_on=['DateOfDeparture', 'Arrival'],sort=False) 

        # Finally getting rid of departure, arrival, and date columns now that we do not need them to merge
        X_encoded = X_encoded.drop('Departure', axis=1)
        X_encoded = X_encoded.drop('Arrival', axis=1)
        X_encoded = X_encoded.drop('DateOfDeparture', axis=1)
        X_array = X_encoded.values

In [4]:
# Combined regressor 
from sklearn.base import BaseEstimator
from sklearn.linear_model import Ridge 
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor

class Regressor(BaseEstimator):
    def __init__(self):
        self.reg = RandomForestRegressor(n_estimators=20)
        self.reg2 = Lasso()
        self.reg3 = LassoLars()
        self.reg4 = Ridge()
        self.reg5 = LinearRegression()
        self.reg6 = BayesianRidge()
        self.metareg = RandomForestRegressor(n_estimators=20)

    def fit(self, X, y):
        self.reg.fit(X, y)
        self.reg2.fit(X, y)
        self.reg3.fit(X,y)
        self.reg4.fit(X,y)
        self.reg5.fit(X,y)
        self.reg6.fit(X,y)
        X_combined = np.vstack([self.reg.predict(X), self.reg2.predict(X),self.reg3.predict(X),self.reg4.predict(X),self.reg5.predict(X)]).T
        self.metareg.fit(X_combined, y)


    def predict(self, X):
        pred1 = self.reg.predict(X)
        pred2 = self.reg2.predict(X)
        pred3 = self.reg3.predict(X)
        pred4 = self.reg4.predict(X)
        pred5 = self.reg5.predict(X)
        pred6 = self.reg6.predict(X)
        X_combined = np.vstack([pred1, pred2, pred3, pred4, pred5]).T
        return self.metareg.predict(X_combined)

In [5]:
# Baseline regressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator
class Regressor2(BaseEstimator):
    def __init__(self):
        self.reg = RandomForestRegressor(
            n_estimators=20, max_depth=50, max_features=10)

    def fit(self, X, y):
        self.reg.fit(X, y)

    def predict(self, X):
        return self.reg.predict(X)

In [111]:
features = X_encoded
X_columns = features.columns
X_array = features.values

X_train, X_test, y_train, y_test = train_test_split(X_array, y_array, test_size=0.2, random_state=0)

In [113]:
X_encoded.head()

,WeeksToDeparture,std_wtd,d_ManyFlights,a_ManyFlights,d_ATL,d_BOS,d_CLT,d_DEN,d_DFW,d_DTW,d_EWR,d_IAH,d_JFK,d_LAS,d_LAX,d_LGA,d_MCO,d_MIA,d_MSP,d_ORD,d_PHL,d_PHX,d_SEA,d_SFO,a_ATL,a_BOS,a_CLT,a_DEN,a_DFW,a_DTW,a_EWR,a_IAH,a_JFK,a_LAS,a_LAX,a_LGA,a_MCO,a_MIA,a_MSP,a_ORD,a_PHL,a_PHX,a_SEA,a_SFO,year,month,day,weekday,week,n_days,y_2011,y_2012,y_2013,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,m_11,m_12,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,wd_0,wd_1,wd_2,wd_3,wd_4,wd_5,wd_6,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17,w_18,w_19,w_20,w_21,w_22,w_23,w_24,w_25,w_26,w_27,w_28,w_29,w_30,w_31,w_32,w_33,w_34,w_35,w_36,w_37,w_38,w_39,w_40,w_41,w_42,w_43,w_44,w_45,w_46,w_47,w_48,w_49,w_50,w_51,w_52,Max_TemperatureC_x,MeanDew_PointC_x,Max_Humidity_x,Max_Sea_Level_PressurehPa_x,Max_VisibilityKm_x,Mean_VisibilityKm_x,Min_VisibilitykM_x,Max_Wind_SpeedKm/h_x,Mean_Wind_SpeedKm/h_x,Precipitationmm_x,CloudCover_x,WindDirDegrees_x,Rain_x,Thunderstorm_x,Fog_x,Snow_x,Hail_x,Tornado_x,latitude_deg_x,longitude_deg_x,elevation_ft_x,2018_x,2017_x,2016_x,2015_x,Max_TemperatureC_y,MeanDew_PointC_y,Max_Humidity_y,Max_Sea_Level_PressurehPa_y,Max_VisibilityKm_y,Mean_VisibilityKm_y,Min_VisibilitykM_y,Max_Wind_SpeedKm/h_y,Mean_Wind_SpeedKm/h_y,Precipitationmm_y,CloudCover_y,WindDirDegrees_y,Rain_y,Thunderstorm_y,Fog_y,Snow_y,Hail_y,Tornado_y,latitude_deg_y,longitude_deg_y,elevation_ft_y,2018_y,2017_y,2016_y,2015_y
0,12.875000,9.812647,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2012,6,19,1,25,15510,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,35.0,18.0,62.0,1014.0,16.0,16.0,16.0,37.0,25.0,0.0,1.0,208.0,1.0,0.0,0.0,0.0,0.0,0.0,41.978600,-87.904800,672.0,39874879.0,38593028.0,37589899.0,36305668.0,34.0,21.0,82.0,1012.0,16.0,16.0,16.0,48.0,29.0,0.0,5.0,161.0,1.0,0.0,0.0,0.0,0.0,0.0,32.896801,-97.038002,607.0,32800721.0,31816933.0,31283579.0,31589832.0
1,14.285714,9.466734,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2012,9,10,0,37,15593,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,36.0,13.0,51.0,1011.0,16.0,15.0,11.0,32.0,18.0,0.0,5.0,187.0,1.0,0.0,0.0,0.0,0.0,0.0,36.080101,-115.152000,2181.0,23655285.0,23364393.0,22833267.0,21824231.0,33.0,-6.0,21.0,1011.0,16.0,16.0,16.0,35.0,15.0,0.0,3.0,207.0,1.0,0.0,0.0,0.0,0.0,0.0,39.861698,-104.672997,5431.0,31363573.0,29809097.0,28267394.0,26280043.0
2,10.863636,9.035883,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2012,10,5,4,40,15618,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,22.0,16.0,93.0,1018.0,16.0,13.0,8.0,24.0,8.0,0.0,5.0,266.0,0.0,0.0,1.0,0.0,0.0,0.0,33.942501,-118.407997,125.0,42626783.0,41232432.0,39636042.0,36351226.0
3,11.480000,7.990202,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2011,10,9,6,40,15256,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,22.0,10.0,67.0,1026.0,16.0,16.0,16.0,37.0,26.0,0.0,8.0,70.0,1.0,0.0,0.0,0.0,0.0,0.0,33.636700,-84.428101,1026.0,51866464.0,50251964.0,50501858.0,49340732.0,27.0,10.0,83.0,1028.0,16.0,16.0,16.0,23.0,6.0,0.0,1.0,93.0,1.0,0.0,0.0,0.0,0

### Getting the base dataset

In [31]:
problem = imp.load_source('', 'problem.py')
X_df, y_array = problem.get_train_data()

In [32]:
# Exploring the dataset
X_df.isna().sum()
X_df.describe()
X_df['Departure'].unique()
X_df['Arrival'].unique()
X_df.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd
0,2012-06-19,ORD,DFW,12.875000,9.812647
1,2012-09-10,LAS,DEN,14.285714,9.466734
2,2012-10-05,DEN,LAX,10.863636,9.035883
3,2011-10-09,ATL,ORD,11.480000,7.990202
4,2012-02-21,DEN,SFO,11.450000,9.517159


In [33]:
X_df['Departure'].unique()

array(['ORD', 'LAS', 'DEN', 'ATL', 'SFO', 'EWR', 'IAH', 'LAX', 'DFW',
       'SEA', 'JFK', 'PHL', 'MIA', 'DTW', 'BOS', 'MSP', 'CLT', 'MCO',
       'PHX', 'LGA'], dtype=object)

In [34]:
X_df['Arrival'].unique()

array(['DFW', 'DEN', 'LAX', 'ORD', 'SFO', 'MCO', 'LAS', 'CLT', 'MSP',
       'EWR', 'PHX', 'DTW', 'MIA', 'BOS', 'PHL', 'JFK', 'ATL', 'LGA',
       'SEA', 'IAH'], dtype=object)

### Defining the feature extractor (WIP)

In [80]:
# Feature extractor test
X_encoded = X_df
# path = os.path.dirname(__file__)

# Feature engineering
# Creating columns to distinguish between the two main airports for flights and the rest
X_encoded['d_ManyFlights'] = 0
X_encoded['a_ManyFlights'] = 0
X_encoded['d_ManyFlights'][X_df['Departure'] == 'ORD'] = 1
X_encoded['d_ManyFlights'][X_df['Departure'] == 'ATL'] = 1
X_encoded['a_ManyFlights'][X_df['Arrival'] == 'ORD'] = 1
X_encoded['a_ManyFlights'][X_df['Arrival'] == 'ATL'] = 1

# Categorical encoding of departure and arrival airports
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))

# Categorical encoding of the dates 
X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
X_encoded['day'] = X_encoded['DateOfDeparture'].dt.day
X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))



# Auxiliary dataset we will use to complete our data

# external_dataTest= pd.read_csv(os.path.join(path, 'external_dataTest.csv'))
#external_data = external_dataTest['Date', 'AirPort', 'Max TemperatureC', 'Mean TemperatureC',
#       'Min TemperatureC', 'Dew PointC', 'MeanDew PointC', 'Min DewpointC',
#       'Max Humidity', 'Mean Humidity', 'Min Humidity',
#       'Max Sea Level PressurehPa', 'Mean Sea Level PressurehPa',
#       'Min Sea Level PressurehPa', 'Max VisibilityKm', 'Mean VisibilityKm',
#       'Min VisibilitykM', 'Max Wind SpeedKm/h', 'Mean Wind SpeedKm/h',
#       'Max Gust SpeedKm/h', 'Precipitationmm', 'CloudCover', 'Events',
#       'WindDirDegrees', 'Rain', 'Thunderstorm', 'Fog', 'Snow', 'Hail',
#       'Tornado', 'latitude_deg', 'longitude_deg', 'elevation_ft',
#       '2018', '2017', '2016', '2015']

# Now merging external data with out dataset 
# Creating two tables, one for departures, the other for arrivals
external_dataDeparture = external_data.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Departure'})
external_dataArrival = external_data.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})

# Merging them with X_encoded   
X_encoded = pd.merge(X_encoded, external_dataDeparture, how='left',left_on=['DateOfDeparture', 'Departure'],
                             right_on=['DateOfDeparture', 'Departure'],sort=False)
X_encoded = pd.merge(X_encoded, external_dataArrival, how='left',left_on=['DateOfDeparture', 'Arrival'],
                             right_on=['DateOfDeparture', 'Arrival'],sort=False) 

# Finally getting rid of departure, arrival, and date columns now that we do not need them to merge
X_encoded = X_encoded.drop('Departure', axis=1)
X_encoded = X_encoded.drop('Arrival', axis=1)
X_encoded = X_encoded.drop('DateOfDeparture', axis=1)
X_array = X_encoded.values

ValueError: You are trying to merge on datetime64[ns] and object columns. If you wish to proceed you should use pd.concat

In [81]:
X_encoded.head()

,DateOfDeparture,Departure,Arrival,WeeksToDeparture,std_wtd,d_ManyFlights,a_ManyFlights,d_ATL,d_BOS,d_CLT,d_DEN,d_DFW,d_DTW,d_EWR,d_IAH,d_JFK,d_LAS,d_LAX,d_LGA,d_MCO,d_MIA,d_MSP,d_ORD,d_PHL,d_PHX,d_SEA,d_SFO,a_ATL,a_BOS,a_CLT,a_DEN,a_DFW,a_DTW,a_EWR,a_IAH,a_JFK,a_LAS,a_LAX,a_LGA,a_MCO,a_MIA,a_MSP,a_ORD,a_PHL,a_PHX,a_SEA,a_SFO,year,month,day,weekday,week,n_days,y_2011,y_2012,y_2013,m_1,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,m_11,m_12,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,d_11,d_12,d_13,d_14,d_15,d_16,d_17,d_18,d_19,d_20,d_21,d_22,d_23,d_24,d_25,d_26,d_27,d_28,d_29,d_30,d_31,wd_0,wd_1,wd_2,wd_3,wd_4,wd_5,wd_6,w_1,w_2,w_3,w_4,w_5,w_6,w_7,w_8,w_9,w_10,w_11,w_12,w_13,w_14,w_15,w_16,w_17,w_18,w_19,w_20,w_21,w_22,w_23,w_24,w_25,w_26,w_27,w_28,w_29,w_30,w_31,w_32,w_33,w_34,w_35,w_36,w_37,w_38,w_39,w_40,w_41,w_42,w_43,w_44,w_45,w_46,w_47,w_48,w_49,w_50,w_51,w_52
0,2012-06-19,ORD,DFW,12.875000,9.812647,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2012,6,19,1,25,15510,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2012-09-10,LAS,DEN,14.285714,9.466734,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2012,9,10,0,37,15593,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,2012-10-05,DEN,LAX,10.863636,9.035883,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,2012,10,5,4,40,15618,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
3,2011-10-09,ATL,ORD,11.480000,7.990202,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,2011,10,9,6,40,15256,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0
4,2012-02-21,DEN,SFO,11.450000,9.517159,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2012,2,21,1,8,15391,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


### Getting auxiliary datasets

#### External data

In [86]:
data_weather = pd.read_csv("submissions/sarvic_test1/external_data.csv", sep = ",")
data_weather.head()

,Date,AirPort,Max TemperatureC,Mean TemperatureC,Min TemperatureC,Dew PointC,MeanDew PointC,Min DewpointC,Max Humidity,Mean Humidity,Min Humidity,Max Sea Level PressurehPa,Mean Sea Level PressurehPa,Min Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Max Gust SpeedKm/h,Precipitationmm,CloudCover,Events,WindDirDegrees
0,2011-09-01,ATL,35,29,24,21,18,14,79,56,32,1022,1019,1017,16,16,11,19,6,26.0,0.00,3,NaN,129
1,2011-09-02,ATL,36,29,22,17,15,14,61,46,30,1019,1016,1014,16,16,16,24,7,34.0,0.00,2,NaN,185
2,2011-09-03,ATL,35,29,23,17,16,14,64,47,30,1015,1013,1011,16,16,16,19,7,26.0,0.00,4,NaN,147
3,2011-09-04,ATL,27,24,22,22,19,16,93,72,51,1014,1012,1011,16,14,4,21,9,26.0,6.10,6,Rain,139
4,2011-09-05,ATL,26,24,22,23,22,20,94,91,87,1010,1005,999,16,13,3,32,16,45.0,16.00,8,Rain-Thunderstorm,149


In [87]:
data_weather.isna().sum()

Date                             0
AirPort                          0
Max TemperatureC                 0
Mean TemperatureC                0
Min TemperatureC                 0
Dew PointC                       0
MeanDew PointC                   0
Min DewpointC                    0
Max Humidity                     0
Mean Humidity                    0
Min Humidity                     0
Max Sea Level PressurehPa        0
Mean Sea Level PressurehPa       0
Min Sea Level PressurehPa        0
Max VisibilityKm                 0
Mean VisibilityKm                0
Min VisibilitykM                 0
Max Wind SpeedKm/h               0
Mean Wind SpeedKm/h              0
Max Gust SpeedKm/h             290
Precipitationmm                  0
CloudCover                       0
Events                        6745
WindDirDegrees                   0
dtype: int64

In [88]:
# data_weather.profile_report()

In [89]:
# Events: affecting rain to the events we do not know
# Then, categorical encoding of the Events column
data_weather["Events"][data_weather["Events"].isna()] = "Rain"
new = data_weather["Events"].str.split('-',expand=True)
events_list = ["Rain", "Thunderstorm", "Fog", "Snow", "Hail", "Tornado"]
for event in events_list:
    event_bool = [new == event]
    event_bool = event_bool[0]
    data_weather[event] = event_bool.sum(axis = 1)

# Changing the date column to the right format
data_weather['Date'] = pd.to_datetime(data_weather['Date'])

# Getting rid of the "T" values in Precipitationmm and replacing them by a very small value
# Then casting Precipitationmm as floats
data_weather["Precipitationmm"][data_weather["Precipitationmm"]=="T"] = 0.10
data_weather["Precipitationmm"] = data_weather["Precipitationmm"].astype(float)

# Dropping irrelevant, highly correlated columns
data_weather = data_weather.drop(columns = ["Max Gust SpeedKm/h","Mean Humidity","Mean Sea Level PressurehPa",
                                            "Mean TemperatureC", "Dew PointC", "Min DewpointC",
                                            "Min Humidity","Min Sea Level PressurehPa", "Min TemperatureC","Events"])
# Finally getting a look at our dataset
data_weather.head()

,Date,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,Max Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Precipitationmm,CloudCover,WindDirDegrees,Rain,Thunderstorm,Fog,Snow,Hail,Tornado
0,2011-09-01,ATL,35,18,79,1022,16,16,11,19,6,0.0,3,129,1,0,0,0,0,0
1,2011-09-02,ATL,36,15,61,1019,16,16,16,24,7,0.0,2,185,1,0,0,0,0,0
2,2011-09-03,ATL,35,16,64,1015,16,16,16,19,7,0.0,4,147,1,0,0,0,0,0
3,2011-09-04,ATL,27,19,93,1014,16,14,4,21,9,6.1,6,139,1,0,0,0,0,0
4,2011-09-05,ATL,26,22,94,1010,16,13,3,32,16,16.0,8,149,1,1,0,0,0,0


In [90]:
data_weather.isna().sum(axis=0)

Date                         0
AirPort                      0
Max TemperatureC             0
MeanDew PointC               0
Max Humidity                 0
Max Sea Level PressurehPa    0
Max VisibilityKm             0
Mean VisibilityKm            0
Min VisibilitykM             0
Max Wind SpeedKm/h           0
Mean Wind SpeedKm/h          0
Precipitationmm              0
CloudCover                   0
WindDirDegrees               0
Rain                         0
Thunderstorm                 0
Fog                          0
Snow                         0
Hail                         0
Tornado                      0
dtype: int64

#### Airport data

In [91]:
data_geog = pd.read_csv("https://ourairports.com/data/airports.csv", sep = ',',verbose=False)
# Selecting the countries and airports we are interested in  
data_geog = data_geog[data_geog["iso_country"] == "US"]
data_geog = data_geog[data_geog["type"]=="large_airport"]
data_geog.head()

,id,ident,type,name,latitude_deg,longitude_deg,elevation_ft,continent,iso_country,iso_region,municipality,scheduled_service,gps_code,iata_code,local_code,home_link,wikipedia_link,keywords
26255,16091,KABQ,large_airport,Albuquerque International Sunport,35.040199,-106.609001,5355.0,NaN,US,US-NM,Albuquerque,yes,KABQ,ABQ,ABQ,http://www.abqsunport.com/,https://en.wikipedia.org/wiki/Albuquerque_Inte...,NaN
26274,3364,KADW,large_airport,Joint Base Andrews,38.810799,-76.866997,280.0,NaN,US,US-MD,Camp Springs,no,KADW,ADW,ADW,http://www.jba.af.mil/,https://en.wikipedia.org/wiki/Joint_Base_Andrews,Andrews Air Force Base
26287,3366,KAFW,large_airport,Fort Worth Alliance Airport,32.987598,-97.318802,722.0,NaN,US,US-TX,Fort Worth,no,KAFW,AFW,AFW,http://www.allianceairport.com/,https://en.wikipedia.org/wiki/Fort_Worth_Allia...,NaN
26291,3368,KAGS,large_airport,Augusta Regional At Bush Field,33.369900,-81.964500,144.0,NaN,US,US-GA,Augusta,yes,KAGS,AGS,AGS,NaN,https://en.wikipedia.org/wiki/Augusta_Regional...,NaN
26324,3377,KAMA,large_airport,Rick Husband Amarillo International Airport,35.219398,-101.706001,3607.0,NaN,US,US-TX,Amarillo,yes,KAMA,AMA,AMA,http://airport.amarillo.gov/,https://en.wikipedia.org/wiki/Rick_Husband_Ama...,NaN


In [92]:
# Creating a dictionary to map a IATA code to the name of the airport and select only those we want
my_dict1 = {'ORD':"Chicago O'Hare International Airport", 'LAS':'McCarran International Airport', 
 'DEN':'Denver International Airport', 'ATL':'Hartsfield Jackson Atlanta International Airport', 
 'SFO':'San Francisco International Airport', 'EWR':'Newark Liberty International Airport',
 'IAH':'George Bush Intercontinental Houston Airport', 'LAX':'Los Angeles International Airport', 
 'DFW':'Dallas Fort Worth International Airport', 'SEA': 'Seattle Tacoma International Airport', 
 'JFK':'John F Kennedy International Airport', 'PHL':'Philadelphia International Airport', 
 'MIA':'Miami International Airport', 'DTW': 'Detroit Metropolitan Wayne County Airport',
 'BOS':'General Edward Lawrence Logan International Airport', 'MSP': 'Minneapolis-St Paul International/Wold-Chamberlain Airport', 
 'CLT':'Charlotte Douglas International Airport', 'MCO':'Orlando International Airport', 
 'PHX':'Phoenix Sky Harbor International Airport', 'LGA':'La Guardia Airport'}
my_dict2 = {y:x for x,y in my_dict1.items()}

In [93]:
# Selecting only the airports that are in our initial dataset
data_geog['IATA'] = data_geog['name'].map(my_dict2)
data_geog = data_geog[data_geog['IATA'].isna() == False]

In [94]:
data_geog = data_geog.drop(columns = ["id", "ident", "type", "continent", "keywords","gps_code",
                                      "local_code","home_link","wikipedia_link", "iata_code", 
                                      "name", "iso_region", "municipality", "iso_country", "scheduled_service"])
data_geog = data_geog.rename(columns={"IATA":"AirPort"})
data_geog.head()

,latitude_deg,longitude_deg,elevation_ft,AirPort
26376,33.636700,-84.428101,1026.0,ATL
26502,42.364300,-71.005203,20.0,BOS
26663,35.214001,-80.943100,748.0,CLT
26805,39.861698,-104.672997,5431.0,DEN
26810,32.896801,-97.038002,607.0,DFW


In [95]:
external_data = pd.merge(data_weather, data_geog, how='left',left_on=['AirPort'], right_on=['AirPort'],sort=False)
external_data.head()

,Date,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,Max Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Precipitationmm,CloudCover,WindDirDegrees,Rain,Thunderstorm,Fog,Snow,Hail,Tornado,latitude_deg,longitude_deg,elevation_ft
0,2011-09-01,ATL,35,18,79,1022,16,16,11,19,6,0.0,3,129,1,0,0,0,0,0,33.6367,-84.428101,1026.0
1,2011-09-02,ATL,36,15,61,1019,16,16,16,24,7,0.0,2,185,1,0,0,0,0,0,33.6367,-84.428101,1026.0
2,2011-09-03,ATL,35,16,64,1015,16,16,16,19,7,0.0,4,147,1,0,0,0,0,0,33.6367,-84.428101,1026.0
3,2011-09-04,ATL,27,19,93,1014,16,14,4,21,9,6.1,6,139,1,0,0,0,0,0,33.6367,-84.428101,1026.0
4,2011-09-05,ATL,26,22,94,1010,16,13,3,32,16,16.0,8,149,1,1,0,0,0,0,33.6367,-84.428101,1026.0


In [96]:
external_data.isna().sum(axis=0)

Date                         0
AirPort                      0
Max TemperatureC             0
MeanDew PointC               0
Max Humidity                 0
Max Sea Level PressurehPa    0
Max VisibilityKm             0
Mean VisibilityKm            0
Min VisibilitykM             0
Max Wind SpeedKm/h           0
Mean Wind SpeedKm/h          0
Precipitationmm              0
CloudCover                   0
WindDirDegrees               0
Rain                         0
Thunderstorm                 0
Fog                          0
Snow                         0
Hail                         0
Tornado                      0
latitude_deg                 0
longitude_deg                0
elevation_ft                 0
dtype: int64

#### Passenger traffic

In [97]:
data_traffic = pd.read_csv("data/passenger_traff.csv", sep = ",")

In [98]:
data_traffic = data_traffic.drop(columns = ["Airports (large hubs)", "Major city served", "State"])
data_traffic.head()

,Rank_2018,IATA,2018,2017,2016,2015
0,1,ATL,"51,866,464","50,251,964","50,501,858","49,340,732"
1,2,LAX,"42,626,783","41,232,432","39,636,042","36,351,226"
2,3,ORD,"39,874,879","38,593,028","37,589,899","36,305,668"
3,4,DFW,"32,800,721","31,816,933","31,283,579","31,589,832"
4,5,DEN,"31,363,573","29,809,097","28,267,394","26,280,043"


In [99]:
# Cleaning the dataset to use the numbers 
columns = ["Rank_2018", "2018","2017","2016","2015"]
for column in columns: 
    data_traffic.loc[:,column] = data_traffic.loc[:, column].str.replace('\xa0','')
    data_traffic.loc[:,column] = data_traffic.loc[:, column].str.replace(',','')
    data_traffic.loc[:,column] = list(map(int,data_traffic.loc[:,column]))
    
data_traffic.loc[:,"IATA"] = data_traffic.loc[:,"IATA"].str.replace('\xa0','')

# Dropping the rank which we won't use 
data_traffic = data_traffic.drop(columns = ["Rank_2018"])

In [100]:
data_traffic = data_traffic.rename(columns={'IATA': 'AirPort'})

In [101]:
external_data = pd.merge(external_data, data_traffic, how='left',left_on=['AirPort'], right_on=['AirPort'],sort=False)

In [102]:
external_data.head()

,Date,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,Max Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Precipitationmm,CloudCover,WindDirDegrees,Rain,Thunderstorm,Fog,Snow,Hail,Tornado,latitude_deg,longitude_deg,elevation_ft,2018,2017,2016,2015
0,2011-09-01,ATL,35,18,79,1022,16,16,11,19,6,0.0,3,129,1,0,0,0,0,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732
1,2011-09-02,ATL,36,15,61,1019,16,16,16,24,7,0.0,2,185,1,0,0,0,0,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732
2,2011-09-03,ATL,35,16,64,1015,16,16,16,19,7,0.0,4,147,1,0,0,0,0,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732
3,2011-09-04,ATL,27,19,93,1014,16,14,4,21,9,6.1,6,139,1,0,0,0,0,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732
4,2011-09-05,ATL,26,22,94,1010,16,13,3,32,16,16.0,8,149,1,1,0,0,0,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732


In [103]:
external_data.dtypes

Date                         datetime64[ns]
AirPort                              object
Max TemperatureC                      int64
MeanDew PointC                        int64
Max Humidity                          int64
Max Sea Level PressurehPa             int64
Max VisibilityKm                      int64
Mean VisibilityKm                     int64
Min VisibilitykM                      int64
Max Wind SpeedKm/h                    int64
Mean Wind SpeedKm/h                   int64
Precipitationmm                     float64
CloudCover                            int64
WindDirDegrees                        int64
Rain                                  int64
Thunderstorm                          int64
Fog                                   int64
Snow                                  int64
Hail                                  int64
Tornado                               int64
latitude_deg                        float64
longitude_deg                       float64
elevation_ft                    

In [104]:
external_data.isna().sum()

Date                         0
AirPort                      0
Max TemperatureC             0
MeanDew PointC               0
Max Humidity                 0
Max Sea Level PressurehPa    0
Max VisibilityKm             0
Mean VisibilityKm            0
Min VisibilitykM             0
Max Wind SpeedKm/h           0
Mean Wind SpeedKm/h          0
Precipitationmm              0
CloudCover                   0
WindDirDegrees               0
Rain                         0
Thunderstorm                 0
Fog                          0
Snow                         0
Hail                         0
Tornado                      0
latitude_deg                 0
longitude_deg                0
elevation_ft                 0
2018                         0
2017                         0
2016                         0
2015                         0
dtype: int64

In [105]:
external_data_new = external_data.to_csv('external_data.csv', index = None, header=True) 
#Don't forget to add '.csv' at the end of the path
print(external_data_new)

None


### Week-end feature engineering

In [2]:
import pandas as pd
external_data = pd.read_csv("submissions/sarvic_2011_0935/external_data.csv")
external_data.head()

,Date,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,Max Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,...,Tornado,latitude_deg,longitude_deg,elevation_ft,2018,2017,2016,2015,2017GDPPerCapita,interestFlightPrices
0,2011-09-01,ATL,35,18,79,1022,16,16,11,19,...,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0
1,2011-09-02,ATL,36,15,61,1019,16,16,16,24,...,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0
2,2011-09-03,ATL,35,16,64,1015,16,16,16,19,...,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0
3,2011-09-04,ATL,27,19,93,1014,16,14,4,21,...,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0
4,2011-09-05,ATL,26,22,94,1010,16,13,3,32,...,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0


In [3]:
import datetime
import numpy as np 

# Getting the datetime formats for external data
external_data.loc[:,'Date'] = pd.to_datetime(external_data.loc[:,'Date'])    
j = external_data.columns.get_loc("Date")

# Initialising my week-end column
external_data["WeekEnd"] = 0
k = external_data.columns.get_loc("WeekEnd")
# 
n,p = external_data.shape
for i in np.arange(n): 
    date = external_data.iloc[i,j]
    weekno = date.weekday()
    if weekno<5:
        external_data.iloc[i,k] = 0
    else:
        external_data.iloc[i,k] = 1  
external_data.head()

,Date,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,Max Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,...,latitude_deg,longitude_deg,elevation_ft,2018,2017,2016,2015,2017GDPPerCapita,interestFlightPrices,WeekEnd
0,2011-09-01,ATL,35,18,79,1022,16,16,11,19,...,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0,0
1,2011-09-02,ATL,36,15,61,1019,16,16,16,24,...,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0,0
2,2011-09-03,ATL,35,16,64,1015,16,16,16,19,...,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0,1
3,2011-09-04,ATL,27,19,93,1014,16,14,4,21,...,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0,1
4,2011-09-05,ATL,26,22,94,1010,16,13,3,32,...,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0,0


In [4]:
external_data_new = external_data.to_csv('external_data.csv', index = None, header=True) 
#Don't forget to add '.csv' at the end of the path
print(external_data_new)

None


In [ ]:
regressors = [Ridge(),Lasso(),RandomForestRegressor(),LassoLars()]

In [ ]:
class Regressor(BaseEstimator) :
    def __init__(self):
        self.reg = []
        regressors = [Ridge(),Lasso(),RandomForestRegressor(),LassoLars()]
        for regressor in regressors : 
            self.reg.append(regressor)
        self.metareg = RandomForestRegressor(n_estimators=20)  
    
    def fit(self, X, y):
        (n,p) = X.shape
        X_combined = np.empty((1,n))
        for i in np.arange(len(regressors)):
            self.reg[i].fit(X,y)
            X_combined = np.vstack([X_combined, self.reg[i].predict(X)])
        X_combined = X_combined.T
        self.metareg.fit(X_combined, y)
    
    def predict(self, X):
        for i in np.arange(len(regressors)):
            X_combined = np.vstack([X_combined, self.reg[i].predict(X)])
        X_combined = X_combined.T
        return self.metareg.predict(X_combined)

In [ ]:
reg = Regressor()
reg.fit(X_train,y_train)

In [ ]:
X_train.shape

In [ ]:
# Getting the scores on the train data 
scores = cross_val_score(reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error',n_jobs=3)
print("RMSE: {:.4f} +/- {:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

In [ ]:
# Getting the scores on the test data
scores = cross_val_score(reg, X_test, y_test, cv=5, scoring='neg_mean_squared_error')
print("RMSE: {:.4f} +/- {:.4f}".format(np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

In [ ]:
# Regressor 
from sklearn.base import BaseEstimator
from sklearn.linear_model import Ridge 
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoLars
from sklearn.linear_model import BayesianRidge
from sklearn.ensemble import RandomForestRegressor

class Regressor(BaseEstimator):
    def __init__(self):
        self.reg = RandomForestRegressor(n_estimators=20)
        self.reg2 = Lasso()
        self.reg3 = LassoLars()
        self.reg4 = Ridge()
        self.reg5 = LinearRegression()
        self.reg6 = BayesianRidge()
        self.metareg = RandomForestRegressor(n_estimators=20)

    def fit(self, X, y):
        self.reg.fit(X, y)
        self.reg2.fit(X, y)
        self.reg3.fit(X,y)
        self.reg4.fit(X,y)
        self.reg5.fit(X,y)
        X_combined = np.vstack([self.reg.predict(X), self.reg2.predict(X),self.reg3.predict(X),self.reg4.predict(X),self.reg5.predict(X)]).T
        self.metareg.fit(X_combined, y)


    def predict(self, X):
        pred1 = self.reg.predict(X)
        pred2 = self.reg2.predict(X)
        pred3 = self.reg3.predict(X)
        pred4 = self.reg4.predict(X)
        pred5 = self.reg5.predict(X)
        X_combined = np.vstack([pred1, pred2, pred3, pred4, pred5]).T
        return self.metareg.predict(X_combined)

In [ ]:
reg = Regressor()
reg.fit(X_train,y_train)

In [ ]:
# Getting the scores on the train data
scores = cross_val_score(reg, X_train, y_train, cv=5, scoring='neg_mean_squared_error',n_jobs=3)
print("RMSE: {:.4f} +/- {:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

In [ ]:
# Getting the scores on the test data
scores = cross_val_score(reg, X_test, y_test, cv=5, scoring='neg_mean_squared_error')
print("RMSE: {:.4f} +/- {:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

In [ ]:
# Insert baseline regressor here
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator
class Regressor2(BaseEstimator):
    def __init__(self):
        self.reg = RandomForestRegressor(
            n_estimators=20, max_depth=50, max_features=10)

    def fit(self, X, y):
        self.reg.fit(X, y)

    def predict(self, X):
        return self.reg.predict(X)

In [24]:
# Getting the scores on the train data
scores = cross_val_score(reg2, X_train, y_train, cv=5, scoring='neg_mean_squared_error',n_jobs=3)
print("RMSE: {:.4f} +/- {:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

RMSE: 0.6039 +/- 0.0158


In [25]:
# Getting the scores on the test data
scores = cross_val_score(reg2, X_test, y_test, cv=5, scoring='neg_mean_squared_error')
print("RMSE: {:.4f} +/- {:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

RMSE: 0.6962 +/- 0.0212


In [69]:
# Insert test regressor here
from sklearn.ensemble import RandomForestRegressor
from sklearn.base import BaseEstimator
from sklearn.linear_model import LogisticRegression
class Regressor2(BaseEstimator):
    def __init__(self):
        self.reg = LinearRegression()

    def fit(self, X, y):
        self.reg.fit(X, y)

    def predict(self, X):
        return self.reg.predict(X)

In [70]:
reg2 = Regressor2()
reg2.fit(X_train,y_train)

In [71]:
# Getting the scores on the train data
scores = cross_val_score(reg2, X_train, y_train, cv=5, scoring='neg_mean_squared_error',n_jobs=3)
print("RMSE: {:.4f} +/- {:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

RMSE: 0.6298 +/- 0.0059


In [73]:
# Getting the scores on the test data
scores = cross_val_score(reg2, X_test, y_test, cv=5, scoring='neg_mean_squared_error')
print("RMSE: {:.4f} +/- {:.4f}".format(
    np.mean(np.sqrt(-scores)), np.std(np.sqrt(-scores))))

RMSE: 217824.4508 +/- 435647.5722


In [10]:
# Feature extractor 
import pandas as pd
import os


class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_array):
        pass

    def transform(self, X_df):
        X_encoded = X_df
        path = os.path.dirname(__file__)
        data_weather = pd.read_csv(os.path.join(path, 'external_data.csv'))
        X_weather = data_weather[['Date', 'AirPort', 'Max TemperatureC']]
        X_weatherArrival = X_weather.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})
        X_encoded = pd.merge(X_encoded, X_weatherArrival, how='left',left_on=['DateOfDeparture', 'Arrival'],
                             right_on=['DateOfDeparture', 'Arrival'],sort=False)
        
        X_weatherDeparture = X_weather.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Departure'})
        X_encoded = pd.merge(X_encoded, X_weatherDeparture, how='left',left_on=['DateOfDeparture', 'Departure'],
                             right_on=['DateOfDeparture', 'Departure'],sort=False)

        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))
        X_encoded = X_encoded.drop('Departure', axis=1)
        X_encoded = X_encoded.drop('Arrival', axis=1)

        X_encoded = X_encoded.drop('DateOfDeparture', axis=1)
        X_array = X_encoded.values
        return X_array

In [ ]:
# Feature extractor
import pandas as pd
import os

class FeatureExtractor(object):
    def __init__(self):
        pass

    def fit(self, X_df, y_array):
        pass

    def transform(self, X_df):
        X_encoded = X_df
        path = os.path.dirname(__file__)
        # Feature engineering
        # Creating columns to distinguish between the two main airports for flights and the rest
        X_encoded['d_ManyFlights'] = 0
        X_encoded['a_ManyFlights'] = 0
        X_encoded['d_ManyFlights'][X_df['Departure'] == 'ORD'] = 1
        X_encoded['d_ManyFlights'][X_df['Departure'] == 'ATL'] = 1
        X_encoded['a_ManyFlights'][X_df['Arrival'] == 'ORD'] = 1
        X_encoded['a_ManyFlights'][X_df['Arrival'] == 'ATL'] = 1

        # Categorical encoding of departure and arrival airports
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))

        # Categorical encoding of the dates 
        X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
        X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
        X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
        X_encoded['day'] = X_encoded['DateOfDeparture'].dt.day
        X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
        X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
        X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
        X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))



        # Auxiliary dataset we will use to complete our data
        external_dataTest= pd.read_csv(os.path.join(path, 'external_dataTest.csv'))
        external_data = external_dataTest['Date', 'AirPort', 'Max TemperatureC', 'Mean TemperatureC',
                                          'Min TemperatureC', 'Dew PointC', 'MeanDew PointC', 'Min DewpointC',
                                          'Max Humidity', 'Mean Humidity', 'Min Humidity',
                                          'Max Sea Level PressurehPa', 'Mean Sea Level PressurehPa',
                                          'Min Sea Level PressurehPa', 'Max VisibilityKm', 'Mean VisibilityKm',
                                          'Min VisibilitykM', 'Max Wind SpeedKm/h', 'Mean Wind SpeedKm/h',
                                          'Max Gust SpeedKm/h', 'Precipitationmm', 'CloudCover', 'Events',
                                          'WindDirDegrees', 'Rain', 'Thunderstorm', 'Fog', 'Snow', 'Hail',
                                          'Tornado', 'a_latitude_deg', 'a_longitude_deg', 'a_elevation_ft',
                                          '2018', '2017', '2016', '2015']

        # Now merging external data with out dataset 
        # Creating two tables, one for departures, the other for arrivals
        external_dataDeparture = external_data.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Departure'})
        external_dataArrival = external_data.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})

        # Merging them with X_encoded   
        X_encoded = pd.merge(X_encoded, external_dataDeparture, how='left',left_on=['DateOfDeparture', 'Departure'],
                             right_on=['DateOfDeparture', 'Departure'],sort=False)
        X_encoded = pd.merge(X_encoded, external_dataArrival, how='left',left_on=['DateOfDeparture', 'Arrival'],
                             right_on=['DateOfDeparture', 'Arrival'],sort=False) 

        # Finally getting rid of departure, arrival, and date columns now that we do not need them to merge
        X_encoded = X_encoded.drop('Departure', axis=1)
        X_encoded = X_encoded.drop('Arrival', axis=1)
        X_encoded = X_encoded.drop('DateOfDeparture', axis=1)
        X_array = X_encoded.values

In [ ]:
import os
import glob

os.chdir("/mydir")
extension = 'csv'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]

#combine all files in the list
combined_csv = pd.concat([pd.read_csv(f) for f in all_filenames ])

#export to csv
combined_csv.to_csv( "combined_csv.csv", index=False, encoding='utf-8-sig')



In [118]:
# External data processing
external_data = pd.read_csv('external_data.csv')
external_data["Date"] = pd.to_datetime(external_data["Date"])
external_dataDeparture = external_data.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Departure'})
external_dataArrival = external_data.rename(columns={'Date': 'DateOfDeparture', 'AirPort': 'Arrival'})

# Merging them with X_encoded 
X_encoded = X_df
X_encoded['DateOfDeparture'] = pd.to_datetime(X_encoded['DateOfDeparture'])
X_encoded = pd.merge(X_encoded, external_dataDeparture, how='left',left_on=['DateOfDeparture', 'Departure'],
                             right_on=['DateOfDeparture', 'Departure'],sort=False)
X_encoded = pd.merge(X_encoded, external_dataArrival, how='left',left_on=['DateOfDeparture', 'Arrival'],
                             right_on=['DateOfDeparture', 'Arrival'],sort=False) 


# Feature engineering
# Creating columns to distinguish between the two main airports for flights and the rest
X_encoded['d_ManyFlights'] = 0
X_encoded['a_ManyFlights'] = 0
X_encoded['d_ManyFlights'][X_df['Departure'] == 'ORD'] = 1
X_encoded['d_ManyFlights'][X_df['Departure'] == 'ATL'] = 1
X_encoded['a_ManyFlights'][X_df['Arrival'] == 'ORD'] = 1
X_encoded['a_ManyFlights'][X_df['Arrival'] == 'ATL'] = 1

# Categorical encoding of departure and arrival airports
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Departure'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['Arrival'], prefix='a'))

# Categorical encoding of the dates 
X_encoded['year'] = X_encoded['DateOfDeparture'].dt.year
X_encoded['month'] = X_encoded['DateOfDeparture'].dt.month
X_encoded['day'] = X_encoded['DateOfDeparture'].dt.day
X_encoded['weekday'] = X_encoded['DateOfDeparture'].dt.weekday
X_encoded['week'] = X_encoded['DateOfDeparture'].dt.week
X_encoded['n_days'] = X_encoded['DateOfDeparture'].apply(lambda date: (date - pd.to_datetime("1970-01-01")).days)

X_encoded = X_encoded.join(pd.get_dummies(X_encoded['year'], prefix='y'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['month'], prefix='m'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['day'], prefix='d'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['weekday'], prefix='wd'))
X_encoded = X_encoded.join(pd.get_dummies(X_encoded['week'], prefix='w'))


# Finally getting rid of departure, arrival, and date columns now that we do not need them to merge
X_encoded = X_encoded.drop('Departure', axis=1)
X_encoded = X_encoded.drop('Arrival', axis=1)
X_encoded = X_encoded.drop('DateOfDeparture',axis = 1)
X_array = X_encoded.values

### Creating my external data table with GDPperCapita

In [42]:
gdp = pd.read_csv("data/gdp.csv", sep = ",")
external_data = pd.read_csv("external_data.csv")

In [43]:
data_gdp = gdp.rename(columns={"IATA":"AirPort"})

In [44]:
external_data = pd.merge(external_data, data_gdp, how='left',left_on=['AirPort'],
                         right_on=['AirPort'],sort=False)

In [45]:
external_data_new = external_data.to_csv('external_data.csv', index = None, header=True) 
#Don't forget to add '.csv' at the end of the path
print(external_data_new)

None


### Creating my external data table with interest in flight prices over time

In [8]:
external_data = pd.read_csv("external_data.csv")
external_data = external_data.drop("2017GDPPerCapita_y", axis=1)
external_data = external_data.rename(columns = {"2017GDPPerCapita_x":"2017GDPPerCapita"})
external_data.head()

,Date,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,Max Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,...,Tornado,latitude_deg,longitude_deg,elevation_ft,2018,2017,2016,2015,2017GDPPerCapita,interestFlightPrices
0,2011-09-01,ATL,35,18,79,1022,16,16,11,19,...,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0
1,2011-09-02,ATL,36,15,61,1019,16,16,16,24,...,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0
2,2011-09-03,ATL,35,16,64,1015,16,16,16,19,...,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0
3,2011-09-04,ATL,27,19,93,1014,16,14,4,21,...,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0
4,2011-09-05,ATL,26,22,94,1010,16,13,3,32,...,0,33.6367,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.0,80.0


In [47]:
data_flightPricesInterest = pd.read_csv("data/ggtrendsFlightPrices.csv")
data_flightPricesInterest = data_flightPricesInterest.rename(columns={"Week":"Date"})
data_flightPricesInterest.head()

,Date,interestFlightPrices
0,09/01/2011,80
1,16/01/2011,75
2,23/01/2011,68
3,30/01/2011,52
4,06/02/2011,69


In [48]:
data_flightPricesInterest["Date"] = pd.to_datetime(data_flightPricesInterest["Date"])
#data_flightPricesInterest.head()

In [49]:
external_data["Date"] = pd.to_datetime(external_data["Date"])

In [50]:
external_data = pd.merge(external_data, data_flightPricesInterest, how='left',left_on=['Date'],
                         right_on=['Date'],sort=False)
external_data

,Date,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,Max Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,Mean Wind SpeedKm/h,Precipitationmm,CloudCover,WindDirDegrees,Rain,Thunderstorm,Fog,Snow,Hail,Tornado,latitude_deg,longitude_deg,elevation_ft,2018,2017,2016,2015,2017GDPPerCapita_x,2017GDPPerCapita_y,interestFlightPrices
0,2011-09-01,ATL,35,18,79,1022,16,16,11,19,6,0.00,3,129,1,0,0,0,0,0,33.636700,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.00000,56840.00000,80.0
1,2011-09-02,ATL,36,15,61,1019,16,16,16,24,7,0.00,2,185,1,0,0,0,0,0,33.636700,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.00000,56840.00000,NaN
2,2011-09-03,ATL,35,16,64,1015,16,16,16,19,7,0.00,4,147,1,0,0,0,0,0,33.636700,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.00000,56840.00000,NaN
3,2011-09-04,ATL,27,19,93,1014,16,14,4,21,9,6.10,6,139,1,0,0,0,0,0,33.636700,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.00000,56840.00000,NaN
4,2011-09-05,ATL,26,22,94,1010,16,13,3,32,16,16.00,8,149,1,1,0,0,0,0,33.636700,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.00000,56840.00000,NaN
5,2011-09-06,ATL,24,15,90,1011,16,12,4,47,18,1.52,8,212,1,0,0,0,0,0,33.636700,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.00000,56840.00000,NaN
6,2011-09-07,ATL,21,14,93,1016,16,13,2,23,13,0.10,8,297,1,0,0,0,0,0,33.636700,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.00000,56840.00000,NaN
7,2011-09-08,ATL,23,13,87,1015,16,16,16,27,15,0.00,5,322,1,0,0,0,0,0,33.636700,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.00000,56840.00000,NaN
8,2011-09-09,ATL,27,12,93,1015,16,16,16,26,11,0.00,1,302,1,0,0,0,0,0,33.636700,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.00000,56840.00000,NaN
9,2011-09-10,ATL,29,11,84,1019,16,16,16,21,9,0.00,0,321,1,0,0,0,0,0,33.636700,-84.428101,1026.0,51866464,50251964,50501858,49340732,56840.00000,56840.00000,51.0


In [68]:
# Rebalancing an interest value for each day around the last given data point we have
import math
(n,p) = external_data.shape
last_true_value = external_data.iloc[0,29]
for row in np.arange(n):
    if math.isnan(external_data.iloc[row,29]) :
        external_data.iloc[row,29] = last_true_value
    else:
        external_data.iloc[row,29] = external_data.iloc[row,29]
        last_true_value = external_data.iloc[row,29]

In [9]:
external_data_new = external_data.to_csv('external_data.csv', index = None, header=True) 
#Don't forget to add '.csv' at the end of the path
print(external_data_new)

None


In [4]:
vic

,Date,AirPort,Max TemperatureC,MeanDew PointC,Max Humidity,Max Sea Level PressurehPa,Max VisibilityKm,Mean VisibilityKm,Min VisibilitykM,Max Wind SpeedKm/h,...,2017,2016,2015,2017GDPPerCapita,Weekday,Weekend,Weekend_approx,Busiest_top5,National_holidays,State_holidays
0,2011-09-01,ATL,35,18,79,1022,16,16,11,19,...,50251964,50501858,49340732,56840.00000,3,0,0,1,0,0
1,2011-09-02,ATL,36,15,61,1019,16,16,16,24,...,50251964,50501858,49340732,56840.00000,4,0,1,1,0,0
2,2011-09-03,ATL,35,16,64,1015,16,16,16,19,...,50251964,50501858,49340732,56840.00000,5,1,0,1,0,0
3,2011-09-04,ATL,27,19,93,1014,16,14,4,21,...,50251964,50501858,49340732,56840.00000,6,1,1,1,0,0
4,2011-09-05,ATL,26,22,94,1010,16,13,3,32,...,50251964,50501858,49340732,56840.00000,0,0,0,1,1,1
5,2011-09-06,ATL,24,15,90,1011,16,12,4,47,...,50251964,50501858,49340732,56840.00000,1,0,0,1,0,0
6,2011-09-07,ATL,21,14,93,1016,16,13,2,23,...,50251964,50501858,49340732,56840.00000,2,0,0,1,0,0
7,2011-09-08,ATL,23,13,87,1015,16,16,16,27,...,50251964,50501858,49340732,56840.00000,3,0,0,1,0,0
8,2011-09-09,ATL,27,12,93,1015,16,16,16,26,...,50251964,50501858,49340732,56840.00000,4,0,1,1,0,0
9,2011-09-10,ATL,29,11,84,1019,16,16,16,21,...,50251964,50501858,49340732,56840.00000,5,1,0,1,0,0
